In [30]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

In [17]:
model_path = "./distilcamembert-nli-finetuned-01042024"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [18]:
def predict_nli(premise, hypothesis):
    # Encode the premise and hypothesis
    inputs = tokenizer(premise, hypothesis, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Convert logits to probabilities
    probs = torch.softmax(outputs.logits, dim=-1)
    
    # Assuming the order of labels is ['Entailment', 'Contradiction', 'Neutral']
    labels = ['Entailment', 'Contradiction', 'Neutral']
    prediction = labels[probs.argmax().item()]
    
    return prediction, probs[0].tolist()  

In [29]:
premise = "how do I attach a car battery?"
hypothesis = "This query expresses cognitive dissonance."

prediction, probabilities = predict_nli(premise, hypothesis)

print(f"Prediction: {prediction}")
print(f"Probabilities (Entailment, Contradiction, Neutral): {probabilities}")

Prediction: Entailment
Probabilities (Entailment, Contradiction, Neutral): [0.5049161314964294, 0.05499282851815224, 0.4400910437107086]


In [37]:
queries = pd.read_csv("/Users/kras/Documents/Conferences/2024 | SERP Conf. /questions.csv")
queries["Hypothesis"] = "This query contains cognitive dissonance."
queries.rename(columns={"Question":"Premise"},inplace=True)

,Premise,Hypothesis
0,Can I replace a car battery myself?,This query contains cognitive dissonance.
1,How much does it cost to charge a car battery ...,This query contains cognitive dissonance.
2,How often do you need to replace your car batt...,This query contains cognitive dissonance.
3,What is anormal price for ncar batty When shou...,This query contains cognitive dissonance.
4,How much does it cost to charge a battery UK?,This query contains cognitive dissonance.
5,Why are batteries so expensive?,This query contains cognitive dissonance.
6,Why are start stop batteries so expensive?,This query contains cognitive dissonance.
7,What is the best price on a car battery?,This query contains cognitive dissonance.
8,Can you recharge a car battery?,This query contains cognitive dissonance.
9,Can I replace my car battery myself?,This query contains cognitive dissonance.


In [38]:
predictions = queries.apply(lambda row: predict_nli(row['Premise'], row['Hypothesis']), axis=1)
predictions_df = pd.DataFrame(predictions.tolist(), columns=['prediction', 'probability'])
queries = pd.concat([queries, predictions_df], axis=1)
queries

,Premise,Hypothesis,prediction,probability
0,Can I replace a car battery myself?,This query contains cognitive dissonance.,Neutral,"[0.4374217092990875, 0.09837814420461655, 0.46..."
1,How much does it cost to charge a car battery ...,This query contains cognitive dissonance.,Neutral,"[0.34985312819480896, 0.0786212831735611, 0.57..."
2,How often do you need to replace your car batt...,This query contains cognitive dissonance.,Neutral,"[0.3250810503959656, 0.046759940683841705, 0.6..."
3,What is anormal price for ncar batty When shou...,This query contains cognitive dissonance.,Entailment,"[0.5955379605293274, 0.03497648239135742, 0.36..."
4,How much does it cost to charge a battery UK?,This query contains cognitive dissonance.,Neutral,"[0.4486939609050751, 0.07272052764892578, 0.47..."
5,Why are batteries so expensive?,This query contains cognitive dissonance.,Entailment,"[0.417641818523407, 0.2781175673007965, 0.3042..."
6,Why are start stop batteries so expensive?,This query contains cognitive dissonance.,Entailment,"[0.5538944602012634, 0.25555968284606934, 0.19..."
7,What is the best price on a car battery?,This query contains cognitive dissonance.,Neutral,"[0.3573901653289795, 0.09199243783950806, 0.55..."
8,Can you recharge a car battery?,This query contains cognitive dissonance.,Neutral,"[0.11708123981952667, 0.05134980008006096, 0.8..."
9,Can I replace my car battery myself?,This query contains cognitive dissonance.,Entailment,"[0.4341069459915161, 0.13689769804477692, 0.42..."
